<a href="https://colab.research.google.com/github/abdullah-al-jamil/Sentiment-Analysis/blob/main/sentiment_LSTM_(3_class).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try: 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
keras = tf.keras

In [ ]:
import gensim, re
import numpy as np
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/Sentiment.csv -P "/content/drive/My Drive/app"

--2021-01-12 19:02:04--  https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/Sentiment.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1235571 (1.2M) [text/plain]
Saving to: ‘/content/drive/My Drive/app/Sentiment.csv’

Sentiment.csv       100%[===================>]   1.18M  --.-KB/s    in 0.04s   

2021-01-12 19:02:05 (28.0 MB/s) - ‘/content/drive/My Drive/app/Sentiment.csv’ saved [1235571/1235571]



In [ ]:
df = pd.read_csv('/content/drive/My Drive/app/Sentiment.csv', sep=';')
df.head(5)

,id,text,label,score,lan,domain
0,2283,Your are very good. keep it up,1,1,EN,funny_video
1,2286,চাকিপ ক্যানের মুবি রিব্যু বেচি বেচি,1,0,BN,funny_video
2,2289,আপনি বাংলাদেশী এইটা সত্যি অবাক করার বিষয়,1,0,BN,funny_video
3,2296,নতুন নতুন সিমেনার চাই,1,0,BN,funny_video
4,2300,Eto buddhi kibhabe shomvob,2,1,RN,funny_video


In [ ]:
df1 = df[df.lan=='BN']
len(df1)

2797

In [ ]:
df2 = df1.copy()
df2['score'] = np.where(df1['score']==2, 1,  df1['score'])


df3 = df2.copy()
df3['score'] = np.where(df2['score']==-2, -1,  df2['score'])

In [ ]:
!wget https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/stop_words.txt -P "/content/drive/My Drive/app"
with open('/content/drive/My Drive/app/stop_words.txt', "r") as file1:
    stopwords = file1.read()

--2021-01-12 19:02:07--  https://raw.githubusercontent.com/abdullah-al-jamil/Emotion-and-Sentiment-Analysis/main/stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10442 (10K) [text/plain]
Saving to: ‘/content/drive/My Drive/app/stop_words.txt’

stop_words.txt      100%[===================>]  10.20K  --.-KB/s    in 0s      

2021-01-12 19:02:07 (90.8 MB/s) - ‘/content/drive/My Drive/app/stop_words.txt’ saved [10442/10442]



In [ ]:
def pre_process(sentence):
  #remove link
  sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence)
  sentence = re.sub(r'[^\u0980-\u09FF ]+', '', sentence)
  words = sentence.split()
  new_sentence = ''

  for word in words:
    if word not in stopwords:
      new_sentence += (word + " ")
  return new_sentence

sentences = []
temp_sentences = df3['text']

for i in temp_sentences:
  sentences.append(pre_process(i))

df3['text'] = sentences
df3.head()


,id,text,label,score,lan,domain
1,2286,চাকিপ ক্যানের মুবি রিব্যু বেচি বেচি,1,0,BN,funny_video
2,2289,বাংলাদেশী এইটা সত্যি অবাক,1,0,BN,funny_video
3,2296,সিমেনার চাই,1,0,BN,funny_video
8,2318,হালা গোরা হালা,1,-1,BN,funny_video
9,2319,সুপার দুপার,1,1,BN,funny_video


In [ ]:
fisible = []
for sentence in sentences:
  if len(sentence.split())<=30:
    fisible.append(1)
  else:
    fisible.append(0)

df3['fisible'] = fisible
len(df3)

2797

In [ ]:
df4 = df3[df3.fisible==1]
len(df4)

2689

In [ ]:
labels = []
for i in df4['score']:
  labels.append(i)

y = np.asarray(pd.get_dummies(labels))
pd.get_dummies(labels)


,-1,0,1
0,0,1,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,0,1
...,...,...,...
2684,1,0,0
2685,1,0,0
2686,1,0,0
2687,1,0,0


In [ ]:
vocab_size = 1000
embedding_dim = 100
max_length = 30
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df4.text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df4.text)
padded = pad_sequences(sequences,maxlen=max_length, padding='post')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length, trainable=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300, return_sequences=False)),
    tf.keras.layers.Dense(y.shape[1], activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 100)           100000    
_________________________________________________________________
bidirectional (Bidirectional (None, 600)               962400    
_________________________________________________________________
dense (Dense)                (None, 3)                 1803      
Total params: 1,064,203
Trainable params: 1,064,203
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2, shuffle=True)

In [ ]:
num_epochs = 6
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/6
68/68 [==============================] - 19s 228ms/step - loss: 1.0533 - accuracy: 0.4691 - val_loss: 0.9008 - val_accuracy: 0.6004
Epoch 2/6
68/68 [==============================] - 14s 210ms/step - loss: 0.8289 - accuracy: 0.6210 - val_loss: 0.8081 - val_accuracy: 0.6190
Epoch 3/6
68/68 [==============================] - 14s 208ms/step - loss: 0.6556 - accuracy: 0.7291 - val_loss: 0.7796 - val_accuracy: 0.6636
Epoch 4/6
68/68 [==============================] - 14s 210ms/step - loss: 0.5437 - accuracy: 0.7813 - val_loss: 0.8618 - val_accuracy: 0.6561
Epoch 5/6
68/68 [==============================] - 14s 209ms/step - loss: 0.5051 - accuracy: 0.7905 - val_loss: 0.9025 - val_accuracy: 0.6506
Epoch 6/6
68/68 [==============================] - 14s 212ms/step - loss: 0.4727 - accuracy: 0.8029 - val_loss: 0.9640 - val_accuracy: 0.6431


In [ ]:
model.evaluate(X_test, y_test)

17/17 [==============================] - 1s 54ms/step - loss: 0.9640 - accuracy: 0.6431


[0.9640103578567505, 0.643122673034668]

In [ ]:
pred = model.predict(X_test);
pred[0].max()

0.83878773

In [ ]:
for i in range(len(pred)):
  if pred[i][0]==pred[i].max():
    pred[i][0] = 1
    pred[i][1] = 0
    pred[i][2] = 0
  
  elif pred[i][1]==pred[i].max():
    pred[i][0] = 0
    pred[i][1] = 1
    pred[i][2] = 0

  else: 
    pred[i][0] = 0
    pred[i][1] = 0
    pred[i][2] = 1
  
pred[0]


array([1., 0., 0.], dtype=float32)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, pred, average='micro')

0.6431226765799256

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, pred, average='weighted')

0.6296100992878378